* https://x.com/genkuroki/status/1827879067351466096
  * https://www.emuyn.net/stats/enter_and_analyze_two-way_table

In [2]:
.Table <- matrix(c(15, 13, 5, 15), 2, 2, byrow=TRUE)
rownames(.Table) <- c("行1", "行2")
colnames(.Table) <- c("列A", "列B")
print(.Table)

cat("\n#### カイ二乗検定 (Chi-Square test) 補正なし\n")
.Test <- chisq.test(.Table, correct=FALSE)
print(.Test)

cat("### 期待度数\n")
.Test$expected # Expected Counts

cat("\n\n### 各セルのカイ二乗成分\n")
round(.Test$residuals^2, 2) # Chi-square Components

# 効果量の計算

cat("\n# 効果量:\n")
.Test <- suppressWarnings(chisq.test(.Table, correct=FALSE)) # 効果量は補正しないで計算

# φ係数の計算
phi <- sqrt(.Test$statistic / sum(.Table))

cat("φ係数 =", round(phi, 4), "\n")

# Cramer's Vの計算
k <- min(nrow(.Table), ncol(.Table))
V <- sqrt(.Test$statistic / (sum(.Table) * (k - 1)))

cat("Cramer's V =", round(V, 4), "\n")

# コンティンジェンシー係数の計算
C <- sqrt(.Test$statistic / (.Test$statistic + sum(.Table)))
C_max <- sqrt((k - 1) / k)
C_adjusted <- C / C_max

cat("コンティンジェンシー係数 =", round(C, 4), "\n")

cat("調整済みコンティンジェンシー係数 =", round(C_adjusted, 3), "\n")

cat("\n効果指標 (2行2列の場合のみ):\n")

# 共通の信頼区間計算関数
calculate_ci <- function(estimate, se, ci_level = 0.95) {
  z_score <- qnorm((1 + ci_level) / 2)
  ci_lower <- estimate - z_score * se
  ci_upper <- estimate + z_score * se
  interval_width <- ci_upper - ci_lower
  return(list(lower = ci_lower, upper = ci_upper, width = interval_width))
}

# 結果表示関数（有効数字3桁）
indicator_text <- function(name, estimate, ci, p_value) {
  return(sprintf("%s = %s, 95%%信頼区間: [%s, %s], p値: %s", name, format(signif(estimate, 3), scientific = FALSE), format(signif(ci$lower, 3), scientific = FALSE), format(signif(ci$upper, 3), scientific = FALSE), format(signif(p_value, 2), scientific = FALSE)))
}

# オッズ比
odds_ratio <- (.Table[1,1] * .Table[2,2]) / (.Table[1,2] * .Table[2,1])
se_log_odds <- sqrt(sum(1 / .Table))
log_odds_ratio <- log(odds_ratio)
odds_ratio_ci <- calculate_ci(log_odds_ratio, se_log_odds)
odds_ratio_ci <- lapply(odds_ratio_ci, exp)  # 対数スケールから元のスケールに戻す
odds_ratio_p <- 2 * (1 - pnorm(abs(log_odds_ratio / se_log_odds)))
cat(indicator_text("オッズ比", odds_ratio, odds_ratio_ci, odds_ratio_p), "\n")

# 対数オッズ比
log_odds_ratio_ci <- calculate_ci(log_odds_ratio, se_log_odds)
cat(indicator_text("対数オッズ比", log_odds_ratio, log_odds_ratio_ci, odds_ratio_p), "\n")

# リスク比
p1 <- .Table[1,1] / sum(.Table[1,])
p2 <- .Table[2,1] / sum(.Table[2,])
risk_ratio <- p1 / p2
se_log_risk <- sqrt((1 - p1) / (.Table[1,1]) + (1 - p2) / (.Table[2,1]))
log_risk_ratio <- log(risk_ratio)
risk_ratio_ci <- calculate_ci(log_risk_ratio, se_log_risk)
risk_ratio_ci <- lapply(risk_ratio_ci, exp)  # 対数スケールから元のスケールに戻す
risk_ratio_p <- 2 * (1 - pnorm(abs(log_risk_ratio / se_log_risk)))
cat(indicator_text("リスク比", risk_ratio, risk_ratio_ci, risk_ratio_p), "\n")

# 対数リスク比
log_risk_ratio_ci <- calculate_ci(log_risk_ratio, se_log_risk)
cat(indicator_text("対数リスク比", log_risk_ratio, log_risk_ratio_ci, risk_ratio_p), "\n")

# リスク差
risk_diff <- p1 - p2
se_risk_diff <- sqrt(p1 * (1 - p1) / sum(.Table[1,]) + p2 * (1 - p2) / sum(.Table[2,]))
risk_diff_ci <- calculate_ci(risk_diff, se_risk_diff)
risk_diff_p <- 2 * (1 - pnorm(abs(risk_diff / se_risk_diff)))
cat(indicator_text("リスク差", risk_diff, risk_diff_ci, risk_diff_p), "\n")

# P値関数のデータ作成
create_p_value_data <- function(estimate, se, n_points = 1000) {
  theta_range <- seq(estimate - 4*se, estimate + 4*se, length.out = n_points)
  p_values <- 2 * (1 - pnorm(abs(theta_range - estimate) / se))
  
  list(theta = theta_range, p_value = p_values)
}

# 3つの指標のP値関数データを作成
odds_ratio_data <- create_p_value_data(log_odds_ratio, se_log_odds)
risk_ratio_data <- create_p_value_data(log_risk_ratio, se_log_risk)
risk_diff_data <- create_p_value_data(risk_diff, se_risk_diff)

# プロット作成関数
plot_combined_p_value_functions <- function(odds_ratio_data, risk_ratio_data, risk_diff_data) {
  # PNGデバイスを開く
  png("tmp.png", width = 480, height = 480)
  
  # プロットの範囲を設定
  x_range <- range(c(odds_ratio_data$theta, risk_ratio_data$theta, risk_diff_data$theta))
  y_range <- c(0, 1)
  
  # プロットの初期化
  plot(NULL, xlim = x_range, ylim = y_range, xlab = "対数オッズ比/対数リスク比/リスク差", ylab = "P値", main = "P値関数")
  
  # 各指標のP値関数をプロット
  lines(odds_ratio_data$theta, odds_ratio_data$p_value, col = "red", lwd = 2)
  lines(risk_ratio_data$theta, risk_ratio_data$p_value, col = "blue", lwd = 2)
  lines(risk_diff_data$theta, risk_diff_data$p_value, col = "green", lwd = 2)
  
  # 推定値を垂直線で表示
  abline(v = log_odds_ratio, col = "red", lty = 2)
  abline(v = log_risk_ratio, col = "blue", lty = 2)
  abline(v = risk_diff, col = "green", lty = 2)
  
  # 有意水準（0.05）を水平線で表示
  abline(h = 0.05, col = "gray", lty = 2)
  
  # 凡例を追加
  legend("topright", legend = c("対数オッズ比", "対数リスク比", "リスク差"), col = c("red", "blue", "green"), lwd = 2, lty = 1)
  
  # デバイスを閉じる
  invisible(dev.off())
}

# グラフを作成してPNGファイルとして保存
plot_combined_p_value_functions(odds_ratio_data, risk_ratio_data, risk_diff_data)

    列A 列B
行1  15  13
行2   5  15

#### カイ二乗検定 (Chi-Square test) 補正なし

	Pearson's Chi-squared test

data:  .Table
X-squared = 3.9184, df = 1, p-value = 0.04776

### 期待度数


,列A,列B
行1,11.666667,16.33333
行2,8.333333,11.66667




### 各セルのカイ二乗成分


,列A,列B
行1,0.95,0.68
行2,1.33,0.95



# 効果量:
φ係数 = 0.2857 
Cramer's V = 0.2857 
コンティンジェンシー係数 = 0.2747 
調整済みコンティンジェンシー係数 = 0.389 

効果指標 (2行2列の場合のみ):
オッズ比 = 3.46, 95%信頼区間: [0.986, 12.1], p値: 0.053 
対数オッズ比 = 1.24, 95%信頼区間: [-0.0137, 2.5], p値: 0.053 
リスク比 = 2.14, 95%信頼区間: [0.931, 4.93], p値: 0.073 
対数リスク比 = 0.762, 95%信頼区間: [-0.0716, 1.6], p値: 0.073 
リスク差 = 0.286, 95%信頼区間: [0.0209, 0.551], p値: 0.034 
